### 環境設定

In [ ]:
#%pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22 pydantic==2.10.6

In [2]:
from dotenv import load_dotenv
import os

# .env ファイルを読み込む
path_env="C:\\Users\\ktgu1\\.env"
load_dotenv(path_env)

# 環境変数を取得
langsmith_tracing_v2 = os.getenv("LANGCHAIN_TRACING_V2")
langsmith_endpoint = os.getenv("LANGSMITH_ENDPOINT")
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
langsmith_project = os.getenv("LANGSMITH_PROJECT")

# Langsmithがただしく読み込めているか確認。なぜかtracingがNoneだができてる
print(f"LANGSMITH_TRACING_V2: {langsmith_tracing_v2}")
print(f"LANGSMITH_ENDPOINT: {langsmith_endpoint}")
print(f"LANGSMITH_PROJECT: {langsmith_project}")

LANGSMITH_TRACING_V2: None
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: agent-book


In [3]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください。"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

In [4]:
import operator
from typing import Annotated
from pydantic import BaseModel, Field


class State(BaseModel):
    query: str = Field(..., description="ユーザーからの質問")
    current_role: str = Field(
        default="", description="選定された回答ロール"
    )
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="回答履歴"
    )
    current_judge: bool = Field(
        default=False, description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="", description="品質チェックの判定理由"
    )

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
# 後からmax_tokensの値を変更できるように、変更可能なフィールドを宣言
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [6]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
"""質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )
    # 選択肢の番号のみを返すことを期待したいため、max_tokensの値を1に変更
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [7]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
"""あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問: {query}

回答:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [8]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="判定結果")
    reason: str = Field(default="", description="判定理由")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。
また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [9]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [10]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [11]:
# selectionノードから処理を開始
workflow.set_entry_point("selection")

In [12]:
# selectionノードからansweringノードへ
workflow.add_edge("selection", "answering")
# answeringノードからcheckノードへ
workflow.add_edge("answering", "check")

In [13]:
from langgraph.graph import END

# checkノードから次のノードへの遷移に条件付きエッジを定義
# state.current_judgeの値がTrueならENDノードへ、Falseならselectionノードへ
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [14]:
compiled = workflow.compile()

In [15]:
initial_state = State(query="生成AIについて教えてください")
result = compiled.invoke(initial_state)

In [16]:
result

{'query': '生成AIについて教えてください',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AI製品エキスパートとしてお答えします。\n\n生成AI（生成的人工知能）は、データから新しいコンテンツを生成する能力を持つAI技術の一分野です。これには、テキスト、画像、音声、音楽など、さまざまな形式のコンテンツが含まれます。生成AIの代表的な技術には、以下のようなものがあります。\n\n1. **GPT（Generative Pre-trained Transformer）**: これは、自然言語処理に特化したモデルで、テキストの生成や翻訳、要約などに利用されます。OpenAIのGPTシリーズが有名です。\n\n2. **GAN（Generative Adversarial Networks）**: これは、2つのニューラルネットワークが競い合うことで、よりリアルなデータを生成する技術です。主に画像生成に利用され、フェイク画像の作成や画像のスタイル変換などに使われます。\n\n3. **VAE（Variational Autoencoders）**: これは、データの潜在的な特徴を学習し、新しいデータを生成するためのモデルです。主に画像や音声の生成に利用されます。\n\n生成AIは、クリエイティブなコンテンツの制作、自動化されたカスタマーサービス、医療診断の支援、教育コンテンツのパーソナライズなど、さまざまな分野で応用されています。しかし、倫理的な問題やデータのバイアス、プライバシーの懸念などもあり、これらの課題に対する慎重な対応が求められています。\n\n生成AIは急速に進化しており、今後も多くの分野でその可能性が広がっていくと考えられています。'],
 'current_judge': True,
 'judgement_reason': '回答は生成AIについての基本的な情報を網羅しており、具体例を挙げて技術の説明を行っています。また、生成AIの応用分野や倫理的な課題についても触れており、全体としてバランスの取れた内容です。'}

In [27]:
len(result["messages"])

1

In [30]:
len(result)

5

In [31]:
type(result["messages"])

list

In [29]:
print(result["messages"][0])

生成AI製品エキスパートとしてお答えします。

生成AIとは、人工知能の一分野であり、テキスト、画像、音声、動画などのコンテンツを生成する能力を持つモデルを指します。これらのモデルは、大量のデータを学習し、そのパターンを理解することで、新しいコンテンツを生成することができます。

代表的な生成AIの技術には、以下のようなものがあります。

1. **GPT（Generative Pre-trained Transformer）**: テキスト生成に特化したモデルで、自然言語処理の分野で広く利用されています。GPTは、与えられたプロンプトに基づいて、自然で流暢な文章を生成することができます。

2. **DALL-E**: 画像生成に特化したモデルで、テキストの説明に基づいて新しい画像を生成します。これにより、ユーザーは想像力を駆使して、ユニークなビジュアルコンテンツを作成することができます。

3. **VQ-VAE（Vector Quantized Variational Autoencoder）**: 音声や画像の生成に利用される技術で、データを圧縮し、再構築することで新しいコンテンツを生成します。

生成AIは、クリエイティブな分野だけでなく、ビジネス、教育、医療など多岐にわたる分野で応用されています。例えば、カスタマーサポートの自動化、教育コンテンツのパーソナライズ、医療データの解析などに利用されています。

この技術は急速に進化しており、今後も新しい応用が期待されています。ただし、生成AIの利用にあたっては、倫理的な配慮やデータのプライバシー保護が重要な課題となっています。


In [17]:
print(result["messages"][-1])

生成AI製品エキスパートとしてお答えします。

生成AI（生成的人工知能）は、データから新しいコンテンツを生成する能力を持つAI技術の一分野です。これには、テキスト、画像、音声、音楽など、さまざまな形式のコンテンツが含まれます。生成AIの代表的な技術には、以下のようなものがあります。

1. **GPT（Generative Pre-trained Transformer）**: これは、自然言語処理に特化したモデルで、テキストの生成や翻訳、要約などに利用されます。OpenAIのGPTシリーズが有名です。

2. **GAN（Generative Adversarial Networks）**: これは、2つのニューラルネットワークが競い合うことで、よりリアルなデータを生成する技術です。主に画像生成に利用され、フェイク画像の作成や画像のスタイル変換などに使われます。

3. **VAE（Variational Autoencoders）**: これは、データの潜在的な特徴を学習し、新しいデータを生成するためのモデルです。主に画像や音声の生成に利用されます。

生成AIは、クリエイティブなコンテンツの制作、自動化されたカスタマーサービス、医療診断の支援、教育コンテンツのパーソナライズなど、さまざまな分野で応用されています。しかし、倫理的な問題やデータのバイアス、プライバシーの懸念などもあり、これらの課題に対する慎重な対応が求められています。

生成AIは急速に進化しており、今後も多くの分野でその可能性が広がっていくと考えられています。


In [23]:
print(result["messages"][0])

生成AI製品エキスパートとしてお答えします。

生成AIとは、人工知能の一分野であり、テキスト、画像、音声、動画などのコンテンツを生成する能力を持つモデルを指します。これらのモデルは、大量のデータを学習し、そのパターンを理解することで、新しいコンテンツを生成することができます。

代表的な生成AIの技術には、以下のようなものがあります。

1. **GPT（Generative Pre-trained Transformer）**: テキスト生成に特化したモデルで、自然言語処理の分野で広く利用されています。GPTは、与えられたプロンプトに基づいて、自然で流暢な文章を生成することができます。

2. **DALL-E**: 画像生成に特化したモデルで、テキストの説明に基づいて新しい画像を生成します。これにより、ユーザーは想像力を駆使して、ユニークなビジュアルコンテンツを作成することができます。

3. **VQ-VAE（Vector Quantized Variational Autoencoder）**: 音声や画像の生成に利用される技術で、データを圧縮し、再構築することで新しいコンテンツを生成します。

生成AIは、クリエイティブな分野だけでなく、ビジネス、教育、医療など多岐にわたる分野で応用されています。例えば、カスタマーサポートの自動化、教育コンテンツのパーソナライズ、医療データの解析などに利用されています。

この技術は急速に進化しており、今後も新しい応用が期待されています。ただし、生成AIの利用にあたっては、倫理的な配慮やデータのプライバシー保護が重要な課題となっています。


In [28]:
print(result["messages"][0])

生成AI製品エキスパートとしてお答えします。

生成AIとは、人工知能の一分野であり、テキスト、画像、音声、動画などのコンテンツを生成する能力を持つモデルを指します。これらのモデルは、大量のデータを学習し、そのパターンを理解することで、新しいコンテンツを生成することができます。

代表的な生成AIの技術には、以下のようなものがあります。

1. **GPT（Generative Pre-trained Transformer）**: テキスト生成に特化したモデルで、自然言語処理の分野で広く利用されています。GPTは、与えられたプロンプトに基づいて、自然で流暢な文章を生成することができます。

2. **DALL-E**: 画像生成に特化したモデルで、テキストの説明に基づいて新しい画像を生成します。これにより、ユーザーは想像力を駆使して、ユニークなビジュアルコンテンツを作成することができます。

3. **VQ-VAE（Vector Quantized Variational Autoencoder）**: 音声や画像の生成に利用される技術で、データを圧縮し、再構築することで新しいコンテンツを生成します。

生成AIは、クリエイティブな分野だけでなく、ビジネス、教育、医療など多岐にわたる分野で応用されています。例えば、カスタマーサポートの自動化、教育コンテンツのパーソナライズ、医療データの解析などに利用されています。

この技術は急速に進化しており、今後も新しい応用が期待されています。ただし、生成AIの利用にあたっては、倫理的な配慮やデータのプライバシー保護が重要な課題となっています。


In [26]:
result["messages"]

['生成AI製品エキスパートとしてお答えします。\n\n生成AIとは、人工知能の一分野であり、テキスト、画像、音声、動画などのコンテンツを生成する能力を持つモデルを指します。これらのモデルは、大量のデータを学習し、そのパターンを理解することで、新しいコンテンツを生成することができます。\n\n代表的な生成AIの技術には、以下のようなものがあります。\n\n1. **GPT（Generative Pre-trained Transformer）**: テキスト生成に特化したモデルで、自然言語処理の分野で広く利用されています。GPTは、与えられたプロンプトに基づいて、自然で流暢な文章を生成することができます。\n\n2. **DALL-E**: 画像生成に特化したモデルで、テキストの説明に基づいて新しい画像を生成します。これにより、ユーザーは想像力を駆使して、ユニークなビジュアルコンテンツを作成することができます。\n\n3. **VQ-VAE（Vector Quantized Variational Autoencoder）**: 音声や画像の生成に利用される技術で、データを圧縮し、再構築することで新しいコンテンツを生成します。\n\n生成AIは、クリエイティブな分野だけでなく、ビジネス、教育、医療など多岐にわたる分野で応用されています。例えば、カスタマーサポートの自動化、教育コンテンツのパーソナライズ、医療データの解析などに利用されています。\n\nこの技術は急速に進化しており、今後も新しい応用が期待されています。ただし、生成AIの利用にあたっては、倫理的な配慮やデータのプライバシー保護が重要な課題となっています。']

In [18]:
initial_state = State(query="生成AIについて教えてください")
result = await compiled.ainvoke(initial_state)
result

Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID 57265f1d-3086-47c4-a56b-620f749e6301.')


{'query': '生成AIについて教えてください',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AI製品エキスパートとしてお答えします。\n\n生成AIとは、人工知能の一分野であり、テキスト、画像、音声、動画などのコンテンツを生成する能力を持つモデルを指します。これらのモデルは、大量のデータを学習し、そのパターンを理解することで、新しいコンテンツを生成することができます。\n\n代表的な生成AIの技術には、以下のようなものがあります。\n\n1. **GPT（Generative Pre-trained Transformer）**: テキスト生成に特化したモデルで、自然言語処理の分野で広く利用されています。GPTは、与えられたプロンプトに基づいて、自然で流暢な文章を生成することができます。\n\n2. **DALL-E**: 画像生成に特化したモデルで、テキストの説明に基づいて新しい画像を生成します。これにより、ユーザーは想像力を駆使して、ユニークなビジュアルコンテンツを作成することができます。\n\n3. **VQ-VAE（Vector Quantized Variational Autoencoder）**: 音声や画像の生成に利用される技術で、データを圧縮し、再構築することで新しいコンテンツを生成します。\n\n生成AIは、クリエイティブな分野だけでなく、ビジネス、教育、医療など多岐にわたる分野で応用されています。例えば、カスタマーサポートの自動化、教育コンテンツのパーソナライズ、医療データの解析などに利用されています。\n\nこの技術は急速に進化しており、今後も新しい応用が期待されています。ただし、生成AIの利用にあたっては、倫理的な配慮やデータのプライバシー保護が重要な課題となっています。'],
 'current_judge': True,
 'judgement_reason': '回答は生成AIについての基本的な情報を網羅しており、具体例を挙げて説明しています。内容は正確であり、生成AIの応用例や倫理的な課題についても触れているため、ユーザーの質問に対して適切な回答となっています。'}

In [21]:
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

'apt-get' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


  Using cached pygraphviz-1.14.tar.gz (106 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [94 lines of output]
      C:\Users\ktgu1\AppData\Local\Temp\pip-build-env-lw3xt8zq\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              **********************************************

In [22]:
from IPython.display import Image

Image(compiled.get_graph().draw_png())

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.